## Experiential Task 1 ##
Patents provide an important mechanism for individuals and organizations to protect intellectual property. They encourage innovation and have been extensively linked to firm value. Given their importance to organizations, researchers in economics, finance, and accounting have explored various factors that contribute to patent awards (e.g., R&D expense, industry membership, competition, etc.) and outcomes attributable to patents (e.g., risk profile, executive turnover, etc.). 

One very popular study in the area, [Kogan, Papanikolaou, Seru, and Stoffman (*QJE* 2017; hereafter KPSS)](https://academic.oup.com/qje/article-abstract/132/2/665/3076284), quantify the value created by individual patents and show their measure positively predicts future firm growth. 

In this task, you will use the data provided by KPSS (**KPSS_2022.csv**) and explore how certain features of patents correlate with patent value. Specific requirements include the following:
1. Load the KPSS data and restrict it to patent grants approved in January of 2019.
2. Merge in the company names using the "permno" field and the "crsp_names.csv" dataset. Comment on which organization has the highest average value per patent.
3. Obtain the original patent grants for these patents from the USPTO using the Bulkdata API.
4. Generate a document-term matrix from the patent "abstract" (one paragraph summary about patent) and report the 25 most commonly used words and phrases. Use the following criteria for preprocessing and tokenizing your data: 
    - Include only those tokens that are all letters (alpha) and use lowercase for everything
    - Allow for single words and bigrams
    - Require tokens be at least 3 characters long
    - Exclude stop words
    - Restrict the matrix to the 1,000 most common words
5. Identify and report the 10 terms that correlate most positively and 10 terms that correlate most negatively with patent value ('xi_real' from the KPSS data)

### STEP 1: Load KPSS data & restrict to 2019
Use pandas to load the KPSS data into a data frame called `df`:

In [1]:
import pandas as pd

# Load the KPSS data from the CSV file
df = pd.read_csv('KPSS_2022.csv')



/scratch/656887/ipykernel_3200171/438414030.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


The column `issue_date` is not in a datetime format, so we'll use `pd.todatetime` to convert it. Note that because the number is an 8 digit number, we need to provide the format. I've provided you that code:

In [2]:
df['issue_date'] = pd.to_datetime(df['issue_date'],format="%Y%m%d")

Now restrict the original dataframe, `df`, to only those patents with issue dates in January of 2019. Call this new dataframe `sub`.

In [3]:
# Filter the data for patents granted in Jan 2019
sub = df[(df['issue_date'].dt.year == 2019) & (df['issue_date'].dt.month == 1)]

sub.head()

,patent_num,permno,issue_date,filing_date,xi_nominal,xi_real,cites
2787418,10172273,77520,2019-01-08,20170202.0,26.409036,10.030283,0
2787419,10172275,14144,2019-01-08,20161229.0,9.621185,3.654174,2
2787420,10172291,14144,2019-01-08,20170116.0,9.621185,3.654174,0
2787421,10172277,14144,2019-01-08,20161006.0,9.621185,3.654174,6
2787422,10172280,14144,2019-01-08,20140429.0,9.621185,3.654174,0


### STEP 2: Merge in Company Names and report high values
The data from KPSS has a field called `permno` which allows us to link to a dataset called CRSP ("Center for Research in Security Prices"). I've provided the "names" file for you, so you can merge in organization names to the patent data.

Here's the code you'll need to load the dataset, assuming "crsp_names.csv" is in the same folder as this demo. Label use the variable name `names` for this dataset:

In [4]:
## Load Data:
names = pd.read_csv('crsp_names.csv')
names # examine dataset

,DATE,COMNAM,PERMNO,PERMCO
0,1986-01-07,OPTIMUM MANUFACTURING INC,10000,7952
1,1986-01-09,GREAT FALLS GAS CO,10001,7953
2,1993-11-22,ENERGY WEST INC,10001,7953
3,2009-08-04,ENERGY INC,10001,7953
4,2010-07-09,GAS NATURAL INC,10001,7953
...,...,...,...,...
49736,2013-04-10,VOLTARI CORP,93433,53451
49737,2010-06-14,S & W SEED CO,93434,53427
49738,2010-06-14,SINO CLEAN ENERGY INC,93435,53452
49739,2010-06-29,TESLA MOTORS INC,93436,53453


This file has a history of names for each `PERMNO`, as you should be able to observe. Since our data is relatively recent, drop duplicate values by `PERMNO`, retaining the **last** company name associated with each `permno` (HINT: you should use `drop_duplicates()` for this; see __[documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)__ for how to use and how to retain the final observation).

In [5]:
names = names.sort_values(by='DATE').drop_duplicates(subset='PERMNO', keep='last')

# Lowercase column names so names match for merging later
names.columns = names.columns.str.lower()
names

,date,comnam,permno,permco
4791,1925-12-31,ONYX HOSIERY INC,13629,22457
3647,1925-12-31,INTERNATIONAL SHOE CO,12597,20987
679,1925-12-31,AMERICAN SUMATRA TOB CO,10444,22192
3636,1925-12-31,INTERNATIONAL PAPER CO OLD,12589,56225
1377,1925-12-31,BUTTE COPPER & ZINC CO,10954,22237
...,...,...,...,...
15990,2022-12-29,COYA THERAPEUTICS INC,23615,59456
47538,2022-12-30,BAIJIAYUN GROUP LTD,91664,51551
13292,2022-12-30,RAIN ONCOLOGY INC,21089,58216
15978,2022-12-30,TIDAL TRUST II,23603,59396


Now, merge this data into your `sub` dataset (you only need to merge "COMNAM", but you're free to keep the other columns if you wish). I recommend using `pd.merge`. Call your new dataset `sub2`:

In [6]:
sub2 = sub.merge(names[['permno', 'comnam']], on='permno', how='left')
sub2

,patent_num,permno,issue_date,filing_date,xi_nominal,xi_real,cites,comnam
0,10172273,77520,2019-01-08,20170202.0,26.409036,10.030283,0,A G C O CORP
1,10172275,14144,2019-01-08,20161229.0,9.621185,3.654174,2,C N H INDUSTRIAL N V
2,10172291,14144,2019-01-08,20170116.0,9.621185,3.654174,0,C N H INDUSTRIAL N V
3,10172277,14144,2019-01-08,20161006.0,9.621185,3.654174,6,C N H INDUSTRIAL N V
4,10172280,14144,2019-01-08,20140429.0,9.621185,3.654174,0,C N H INDUSTRIAL N V
...,...,...,...,...,...,...,...,...
6114,10194506,84788,2019-01-29,20180405.0,168.984771,64.181254,0,AMAZON COM INC
6115,10194511,92469,2019-01-29,20170731.0,2.362142,0.897153,2,ECHOSTAR CORP
6116,10194514,84381,2019-01-29,20151111.0,28.048238,10.652860,0,ROCKWELL AUTOMATION INC
6117,10194482,59328,2019-01-29,20171212.0,22.343084,8.486014,0,INTEL CORP


Now, report the 20 companies with higher average value per patent. Use "xi_real" to measure value:

In [7]:
# Group by company name and calculate the average xi_real value
average_values = sub2.groupby('comnam')['xi_real'].mean().reset_index()

# Sort the companies by average xi_real value in descending order 
sorted_values = average_values.sort_values(by='xi_real', ascending=False)

# Select the top 20 companies
top_20 = sorted_values.head(20)

print(top_20)


                            comnam     xi_real
326                    NETFLIX INC  364.899130
254            JPMORGAN CHASE & CO  285.497544
364                    PEPSICO INC  273.411966
92                    BROADCOM INC  272.633542
342                    NVIDIA CORP  215.235192
275                 LILLY ELI & CO  211.978281
80            BANK OF AMERICA CORP  208.387378
10                      ABBVIE INC  191.632884
277                      LINDE PLC  170.863245
421                 STARBUCKS CORP  151.072365
140                 DISNEY WALT CO  150.115539
284            LOWES COMPANIES INC  145.945266
32       ALIBABA GROUP HOLDING LTD  136.766018
169               EXXON MOBIL CORP  129.484244
480   WALGREENS BOOTS ALLIANCE INC  127.495303
202            GILEAD SCIENCES INC  121.594688
367                     PFIZER INC  121.375057
111               CHEVRON CORP NEW  120.859586
470     VERTEX PHARMACEUTICALS INC  108.523734
387  REGENERON PHARMACEUTICALS INC  102.237545


### STEP 3: Use an API to acquire patent details
We're going to use a relatively new API to access the full details of the patent data. The endpoint for this API is `https://developer.uspto.gov/ibd-api/v1/application/grants`. You can review the syntax for the endppoint __[here](https://developer.uspto.gov/ibd-api/swagger-ui/index.html)__.

There are a variety of options for querying data, including simple keyword searches, abstract text, etc. Since we have that patent number in our dataset (`patent_num`), we're going to using the "patentNumber". For instance, the URL for searching for patent number 10172275 is `https://developer.uspto.gov/ibd-api/v1/application/grants?patentNumber=10172275`.

Acquire that one patent and inspect the results (NOTE: You should use `requests` for this, which I've set up for you below, with the keyword `verify` set to False. I've also included some code to suppress warnings.):

In [8]:
import requests
requests.packages.urllib3.disable_warnings()

address = 'https://developer.uspto.gov/ibd-api/v1/application/grants' # add string with address here
page = requests.get(address, verify=False)
print(page.status_code)
# page.text

# Define the API endpoint and the patent number
patent_number = '10172275'
patent_address = f'https://developer.uspto.gov/ibd-api/v1/application/grants?patentNumber={patent_number}'

# Make the API request
patent_page = requests.get(patent_address, verify = False)

# Check the status code of the response 
print(patent_page.status_code)

# Acquire the patent 10172275 and inspect the results
patent_page.text

200
200


'{"results":[{"inventionSubjectMatterCategory":"utility","patentApplicationNumber":"US15394198","filingDate":"12-29-2016","mainCPCSymbolText":"A01B63/16","furtherCPCSymbolArrayText":["A01B49/06","A01B51/04","A01B63/22"],"inventorNameArrayText":["Totten Kip","Boriack Cale","Anderson Brian J.","Prickel Marvin A."],"abstractText":["In one embodiment, an agricultural implement system includes a pivotable lift assembly. The pivotable lift assembly includes a first bar member and a second bar member rotatively coupled to the first bar member. The pivotable lift assembly further includes a first wheel assembly disposed on a first end of the second bar member and a second wheel assembly disposed on a second end of the second bar member. The pivotable lift assembly also includes an attachment assembly configured to attach the pivotable lift assembly to an agricultural implement, wherein the pivotable lift assembly is configured to aid in carrying a weight of the agricultural implement."],"assig

For our analysis, we need "abstractText", which you should be able to see in the raw JSON file (feel free to use ctrl-F to find). Determine how to access this field and print the results below:

In [9]:
import json

# Check the status code
if patent_page.status_code == 200:
    
    # Parse the JSON response
    data = patent_page.json()
    
    # Access the abstractText field
    abstract_text = data['results'][0]['abstractText']
    
    # Print the abstractText
    print(abstract_text)
    
else: 
    print(f"Unable to retrieve data.  Status code: {page.status_code}")
    

['In one embodiment, an agricultural implement system includes a pivotable lift assembly. The pivotable lift assembly includes a first bar member and a second bar member rotatively coupled to the first bar member. The pivotable lift assembly further includes a first wheel assembly disposed on a first end of the second bar member and a second wheel assembly disposed on a second end of the second bar member. The pivotable lift assembly also includes an attachment assembly configured to attach the pivotable lift assembly to an agricultural implement, wherein the pivotable lift assembly is configured to aid in carrying a weight of the agricultural implement.']


In a moment, we're going to collect all of the patent data, but the documentation suggests you can collect more than 1 patent at a time. Instead of making thousands of individual queries, let's obtain the data in blocks of 100 patents. To do this, we should:
1. Create a list of lists, where each inner element is a list of 100 elements
2. Use `",".join()` to convert each list to a string when generating the webquery

There are a number of ways you could go about generating the list of lists. I'll save you some time and provide you with one line of code that takes care of this with list comprehension and `range()`:

In [10]:
# creates list of lists, where inner elements are each 100 patents (except last one)
patents = [sub.iloc[i:i+100]['patent_num'].astype(str).values.tolist() for i in range(0, len(sub), 100)]
len(patents)


62

So we have 62 groups to search, or 62 separate queries.

To finish up **Step 3**, fill in the code below to collect the data. I've set up a list to collect the data:

In [11]:
import time

records = [] # will use to collect the data

for i,patent_group in enumerate(patents):
    print(i+1) # counter to monitor status
    
    # Create a comma-separated string with the contents of patent_group
    pat_group = ','.join(patent_group) # add answer here
    
    # set up the URL you will use to access the data (I encourage you to use an f-string)
    url = f'https://developer.uspto.gov/ibd-api/v1/application/grants?patentNumber={pat_group}' # add answer here
    
    # use requests to get the URL
    page = requests.get(url,verify=False)
    
    # parse with JSON:
    response = page.json() # add answer here
    
    for record in response['results']:
        patent_number = record['patentNumber']
        abstract = record['abstractText'] # add how you will access the text of the abstract
        records.append({'patent_number':patent_number,
                        'abstract':abstract
                       })
        
    time.sleep(1) # Give the USPTO a short break (1 second)      
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62


Now create a dataframe from the data you just collected:

In [12]:
patent_abstracts = pd.DataFrame(records) # add code here to create DataFrame from "records"
patent_abstracts

,patent_number,abstract
0,10172273,[A vehicle control system for controlling the ...
1,10172274,[A vehicle platform comprises a central body t...
2,10172275,"[In one embodiment, an agricultural implement ..."
3,10172276,[An implement frame having a carriage frame fo...
4,10172277,[A port interface for a pneumatic distribution...
...,...,...
6114,10194511,"[Systems, methods, apparatus, and machine-read..."
6115,10194514,[Electrostatic charge grounding is achieved by...
6116,10194518,"[This disclosure describes systems, methods, a..."
6117,10194522,[A method comprises applying an adhesive to a ...


Finally, you can save this data so you don't have to re-run the collection step each time you work on this task:

In [13]:
patent_abstracts.to_csv("./patent_abstracts.csv",sep="^") # using ^ as delimiter to help keep text clean

### STEP 4: Generate Document Term Matrix

In [11]:
# Load your data if needed:
patent_abstracts = pd.read_csv("./patent_abstracts.csv",sep="^")
patent_abstracts

,Unnamed: 0,patent_number,abstract
0,0,10172273,['A vehicle control system for controlling the...
1,1,10172274,['A vehicle platform comprises a central body ...
2,2,10172275,"['In one embodiment, an agricultural implement..."
3,3,10172276,['An implement frame having a carriage frame f...
4,4,10172277,['A port interface for a pneumatic distributio...
...,...,...,...
6114,6114,10194511,"['Systems, methods, apparatus, and machine-rea..."
6115,6115,10194514,['Electrostatic charge grounding is achieved b...
6116,6116,10194518,"['This disclosure describes systems, methods, ..."
6117,6117,10194522,['A method comprises applying an adhesive to a...


Here are your instructions again for how to preprocess the data and generate the DTM. You should use `CountVectorizer`.

<i>Use the following criteria for preprocessing and tokenizing your data: 
- <i>Include only those tokens that are all letters (alpha) and use lowercase for everything
- <i>Allow for single words and bigrams
- <i>Require tokens be at least 3 characters long
- <i>Exclude stop words (Use NLTK for stopwords)
- <i>Restrict the matrix to the 1,000 most common words
    
Use `vec` as the name for your vectorizer and `dtm` as the name for your document-term matrix. Other than that, you may choose to carry out this process by any method you wish. 

In [91]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [92]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [93]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hice1/rwang814/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

# Create DTM:

def preprocess_text(text):
    
    text = text.lower()
    
    # Remove anything that is not alpha or whitespace
    text = re.sub(r'[^a-z\s]', '', text)
    
    return text

# Apply preprocessing to abstract col
patent_abstracts['processed_abstract'] = patent_abstracts['abstract'].apply(lambda x: preprocess_text(eval(x)[0]))

# Set up CountVectorizer
vec = CountVectorizer(
    tokenizer=lambda text: re.findall(r'\b[a-zA-Z]{3,}\b', text),  # Tokenize by regex
    stop_words=stopwords.words('english'),  # Exclude stop words
    ngram_range=(1, 2),  # Include unigrams and bigrams
    max_features=1000  # Limit to the 1,000 most common words
)

# Fit the CountVectorizer and transform the data into a DTM
dtm = vec.fit_transform(patent_abstracts['processed_abstract'])

# Print 
print(f"Vocabulary size: {len(vec.get_feature_names_out())}")
print(dtm.shape)

Vocabulary size: 1000
(6119, 1000)


Now report the 25 most frequently used words:

In [13]:
# Sum occurence of each word
word_freq = np.asarray(dtm.sum(axis=0)).flatten()
terms = vec.get_feature_names_out()

# Create a df for terms and frequencies
freq_df = pd.DataFrame({'term': terms, 'frequency': word_freq})

# Sort df in descending order
freq_df = freq_df.sort_values(by='frequency', ascending=False)

# top 25
top_25_terms = freq_df.head(25)

print(top_25_terms)

            term  frequency
355        first       7854
790       second       5697
251       device       4816
607          one       4767
433     includes       4300
536          may       3959
223         data       3938
884       system       3090
562       method       2906
488        least       2827
669    plurality       2322
76         based       2319
184   configured       2275
830       signal       2043
676      portion       1946
450  information       1925
953         user       1840
489    least one       1786
486        layer       1640
199      control       1597
591      network       1574
716     provided       1555
431      include       1493
422        image       1476
878      surface       1437


#### DataFrame and Sparse Matrix Manipulation Preparation for Step 5: 

In [45]:
# Convert the DTM to a dense matrix and create a DataFrame
dtm_df = pd.DataFrame(dtm.toarray(), columns=vec.get_feature_names_out())

# Replace NaN values with 0
dtm_df.fillna(0, inplace=True)

# print(dtm_df.head())

# convert document term matrix to dense matrix
dense = dtm.todense()

# add vocab words to dense matrix
dtm_df2 = pd.DataFrame(dense, columns=vec.get_feature_names_out())

dtm_df2.head()

,access,accordance,according,account,acid,across,action,active,activity,actuator,...,window,wire,wireless,wireless communication,wireless device,within,without,work,write,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
# Calculate total words 
total_words = dtm_df2.sum(axis=1)
total_words # verify rows = rows in dtm_df

# Divide word count by total words for proportion of words
dtm2 = dtm_df2.divide(total_words, axis=0)
dtm2

# Add xi_real value to matrix
# dtm2['xi_real'] = sub2['xi_real']

,access,accordance,according,account,acid,across,action,active,activity,actuator,...,window,wire,wireless,wireless communication,wireless device,within,without,work,write,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018519,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
# sample_word = dtm2.columns[0]
# pd.concat([dtm2[sample_word], sub2['xi_real']], axis=1).corr()

In [66]:
# access the correlation 
# pd.concat([dtm2[sample_word], sub2['xi_real']], axis=1).corr().values[0,1]

### STEP 5: Which words correlate with value?
The final part of the assignment requires you to correlate each word with value, as measured by `xi_real`. Then, report the 10 words that correlate most positively and most negatively (20 total). I recommend taking the following approach:
1. Write a function that correlates two arrays (or series) and returns the correlation
2. Loop over each word in the vocabulary and store the correlation coefficient in the container of your choosing
3. Create a pandas Series with the final results, where the index of the series is the word.
4. Sort the series in ascending order, and examine the `head` (most negative) and `tail` (most positive)

First, come up with your function to return a correlation between the column of word counts and patent values. Test the function on a sample word.

In [64]:
# Step 1: 
import numpy as np

def mycorr(a,b):
    # fill in function
    
    # dictionary to store each word as a key with the correlation as a value
    corrs = {}
    
    for word in dtm2.columns:
        corr_value = pd.concat([dtm2[word], sub2['xi_real']], axis=1).corr().values[0, 1]
        corrs[word] = corr_value
        
    return corrs # fill in return object

# Test your function
correlations = mycorr(dtm2, sub2)
print(correlations)

{'access': 0.0051432365201352775, 'accordance': -0.010572116748861925, 'according': 0.027153716699501, 'account': 0.01721630937810755, 'acid': 0.029048224278746627, 'across': -0.016245489747086648, 'action': 0.0033695460341063143, 'active': -0.007561227792237327, 'activity': 0.03235454741894677, 'actuator': -0.01206580572122478, 'adapted': -0.004784510237786124, 'addition': 0.002743333533085649, 'additional': 0.017778342008136126, 'additionally': -0.012484267096753768, 'address': -0.000268043419165815, 'adhesive': -0.00666886464958804, 'adjacent': -0.019513160769169917, 'adjust': 0.01343359347142489, 'adjusted': -0.010950804887312445, 'adjusting': -0.012890057881047592, 'adjustment': -0.009512781132694877, 'agent': 0.009686960549737222, 'air': -0.01992426628779771, 'aircraft': -0.0035591269720047602, 'allocation': -0.012042907595619962, 'allow': -0.005294878532134415, 'allows': 0.0077602369033627034, 'along': -0.015101834476535105, 'also': 0.027491919425338608, 'also disclosed': 0.0114

Now complete the rest of  the task using whatever method you prefer (**HINT**: I recommend looping over words in the vocabulary and collecting correlations):

In [82]:
# Step 2: Call function to create dictionary of each word and it's xi_real (patent value) correlation
correlations = mycorr(dtm2, sub2) 

# Step 3: Create a Pandas Series with the final results 
corr_series = pd.Series(correlations)

# Step 4: Sort the series in ascending order 
sorted_corr_series = corr_series.sort_values()


What words exhibit the most negative correlations?

In [85]:
# Examine the head(most negative) and tail(most positive)
print("Words exhibiting the most negative correlations with xi_real (patent value):")
print(sorted_corr_series.head())


Words exhibiting the most negative correlations with xi_real (patent value):
substrate   -0.045299
second      -0.042548
layer       -0.041257
forming     -0.040050
first       -0.039691
dtype: float64


What words exhibit the most positive correlations?

In [86]:
print("\nWords exhibiting the most positive correlations with xi_real (patent value):")
print(sorted_corr_series.tail())


Words exhibiting the most positive correlations with xi_real (patent value):
managing             0.067019
methods              0.070500
relates              0.071810
invention relates    0.072120
making               0.082092
dtype: float64


**FINAL QUESTION**
From this simple analysis, does it appear that the text in the abstract correlates much with value? Do any correlations stand out as particularly intuitive? surprising? Provide a brief response below:

**ANSWER:**
From this simple analysis, it appears that the text in the abstract does not correlate strongly with the patent value (xi_real).  The correlation coefficients are relatively low, indicating that there isn't a strong linear relationship between the words in the abstract and the patent value.  

* Intuitive Correlations: "managing" and "methods" possibly have positive correlations because patents usually involve methods and management processes, that could associate with higher value.  "invention relates" having a positive correlation makes sense since it ties into the purpose or scope of the patent, indicating it's value.  

* Suprising Correlations: "substrate" and "layer" these words could typically be used in patents related to materials or technology.  In certain industries, one might assume these terms to be associated with high value patents.  "first" and "second" strangely are negatively correlated, while these words could be used to describe steps or components in a patent, which could be expected to correlated with a detailed and valuable patent.

While some correlations match with intuition or expectations, the low magnitude of correlation coefficients suggests there are more diverse factors beyond the abstract of a patent's  text that play a critical role in determining patent value.  